# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

Group Number: 37

Group Members:
- Christoper Daniel (18222034)
- Eldwin Pradipta (18222042)
- Theo Livius Natael (18222076)
- Muhammad Ridho Rabbani (18222098)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

from collections import Counter
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

# Import other libraries if needed

## Import Dataset

In [ ]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public


# link drive train.csv: https://drive.google.com/file/d/1UHtTljXs0jkb2PoQyZSSUQtKpcLesTrl/view?usp=drive_link
df = pd.read_csv('https://drive.google.com/uc?id=1UHtTljXs0jkb2PoQyZSSUQtKpcLesTrl')
df.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

print("Splitting data...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

Splitting data...
Training set shape: (112323, 55)
Validation set shape: (28081, 55)


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
def check_missingValues(df):
    missing_values = df.isnull().sum()

    dtypes = df.dtypes
    result = pd.DataFrame({
        'Missing Values': missing_values,
        'Data Type': dtypes
    })

    return result
check_missingValues(df)

,Missing Values,Data Type
id,0,int64
FILENAME,57532,object
URL,43487,object
URLLength,60639,float64
Domain,70197,object
DomainLength,46319,float64
IsDomainIP,42130,float64
TLD,45399,object
CharContinuationRate,48042,float64
TLDLegitimateProb,52873,float64


In [ ]:
imputer = SimpleImputer(strategy='mean')
numerical_features = X_train.select_dtypes(include=np.number).columns
imputer.fit(X_train[numerical_features])

# transform training and validation sets using the fitted imputer
X_train[numerical_features] = imputer.transform(X_train[numerical_features])
X_val[numerical_features] = imputer.transform(X_val[numerical_features])

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
#function clips outliers in the specified columns based on percentile.
def clip_outliers(df, columns, lower_percentile=1, upper_percentile=99):

    #columns= list of columns to clip.
    #lower_percentile= lower percentile for clip.
    #upper_percentile= upper percentile for clip.

    for col in columns:
        lower_bound = df[col].quantile(lower_percentile / 100)
        upper_bound = df[col].quantile(upper_percentile / 100)
        df[col] = np.clip(df[col], lower_bound, upper_bound)
    return df
    #return dataframe with clipped value

#select numerical features for clip
numerical_features = X_train.select_dtypes(include=np.number).columns

#apply clipping function to both training and validation set
X_train = clip_outliers(X_train, numerical_features)
X_val = clip_outliers(X_val, numerical_features)

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
#remove duplicate from training set
X_val.drop_duplicates(inplace=True)

#reset index after removing
X_val.reset_index(drop=True, inplace=True)

#remove duplicates from validation set
X_val.drop_duplicates(inplace=True)

#reset index after removing
X_val.reset_index(drop=True, inplace=True)

#print the shapes of the dataset after remove
print("Training set shape after removing duplicates:", X_val.shape)
print("Validation set shape after removing duplicates:", X_val.shape)

Training set shape after removing duplicates: (28081, 55)
Validation set shape after removing duplicates: (28081, 55)


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
class featureCreator(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.numeric_features = None

    def fit(self, X, y=None):
        if isinstance(X, pd.DataFrame):
            self.numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
        else:
            self.numeric_features = np.arange(X.shape[1])
        return self

    def transform(self, X):
        X_new = X.copy()

        if 'URLLength' in X_new.columns and 'DomainLength' in X_new.columns:
            X_new['URL_Domain_Ratio'] = X_new['URLLength'] / (X_new['DomainLength'] + 1)

        if 'NoOfLettersInURL' in X_new.columns and 'NoOfDegitsInURL' in X_new.columns:
            X_new['Letter_Digit_Ratio'] = X_new['NoOfLettersInURL'] / (X_new['NoOfDegitsInURL'] + 1)

        special_char_cols = ['NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL']
        if all(col in X_new.columns for col in special_char_cols):
            X_new['Total_Special_Chars'] = X_new[special_char_cols].sum(axis=1)

        if 'IsHTTPS' in X_new.columns and 'HasFavicon' in X_new.columns:
            X_new['Security_Score'] = X_new['IsHTTPS'] + X_new['HasFavicon']

        form_cols = ['HasExternalFormSubmit', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField']
        if all(col in X_new.columns for col in form_cols):
            X_new['Form_Complexity'] = X_new[form_cols].sum(axis=1)

        return X_new.values if isinstance(X, np.ndarray) else X_new

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
class featureScaler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.numerical_cols = [col for col in X.columns
                              if X[col].dtype != 'object' and X[col].nunique() > 2 and col != "label"]
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in self.numerical_cols:
            if (X[col] <= 0).any():
                X_transformed[col] = X[col] + 1 - X[col].min()
            X_transformed[col] = np.log(X_transformed[col])
        return X_transformed.values

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
class featureEncoder(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.encoder = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform = 'pandas')

  def fit(self, X, y=None):
    data = X.copy()
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns
    self.encoder.fit(data[categorical_cols])
    return self

  def transform(self, X):
    data = X.copy()
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns

    encoded_data = self.encoder.transform(data[categorical_cols])

    initial_data = X.drop(columns=categorical_cols)
    encoded_df = pd.concat([initial_data, encoded_data], axis=1)
    return encoded_df

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
class ImbalancedDataHandler(BaseEstimator, TransformerMixin):
    def __init__(self, method='smote', random_state=42):
        self.method = method
        self.random_state = random_state

        if self.method == 'smote':
            self.sampler = SMOTE(random_state=self.random_state)
        elif self.method == 'undersample':
            self.sampler = RandomUnderSampler(random_state=self.random_state)
        elif self.method == 'combine':
            self.sampler = SMOTETomek(random_state=self.random_state)
        else:
            raise ValueError("Method must be one of ['smote', 'undersample', 'combine']")

    def fit(self, X, y=None):
        if y is not None:
            self.original_dist = dict(zip(*np.unique(y, return_counts=True)))
        return self

    def transform(self, X, y=None):
        if y is not None:
            X_resampled, y_resampled = self.sampler.fit_resample(X, y)
            self.resampled_dist = dict(zip(*np.unique(y_resampled, return_counts=True)))
            return X_resampled, y_resampled
        return X

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X, y)

    def get_distribution_summary(self):
        if hasattr(self, 'original_dist') and hasattr(self, 'resampled_dist'):
            return {
                'original': self.original_dist,
                'resampled': self.resampled_dist,
                'original_ratio': min(self.original_dist.values()) / max(self.original_dist.values()),
                'resampled_ratio': min(self.resampled_dist.values()) / max(self.resampled_dist.values())
            }
        return None

def analyze_class_imbalance(y):
    unique, counts = np.unique(y, return_counts=True)
    class_counts = dict(zip(unique, counts))
    total = len(y)

    return {
        'class_counts': class_counts,
        'class_proportions': {k: v/total for k, v in class_counts.items()},
        'imbalance_ratio': min(counts) / max(counts)
    }

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [ ]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [ ]:
def preprocess_data(X_train, X_val):
    # Numeric features preprocessing
    numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

    num_imputer = SimpleImputer(strategy='mean')
    num_scaler = StandardScaler()

    cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    cat_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)

    # Fit imputers and encoders on training data
    # Numeric preprocessing
    X_train_numeric = X_train[numeric_features]
    X_val_numeric = X_val[numeric_features]

    X_train_numeric_imputed = num_imputer.fit_transform(X_train_numeric)
    X_val_numeric_imputed = num_imputer.transform(X_val_numeric)

    X_train_numeric_scaled = num_scaler.fit_transform(X_train_numeric_imputed)
    X_val_numeric_scaled = num_scaler.transform(X_val_numeric_imputed)

    # Categorical preprocessing
    X_train_categorical = X_train[categorical_features]
    X_val_categorical = X_val[categorical_features]

    X_train_categorical_imputed = cat_imputer.fit_transform(X_train_categorical)
    X_val_categorical_imputed = cat_imputer.transform(X_val_categorical)

    X_train_categorical_encoded = cat_encoder.fit_transform(X_train_categorical_imputed)
    X_val_categorical_encoded = cat_encoder.transform(X_val_categorical_imputed)

    # Convert numeric scaled to sparse if needed
    if not sp.issparse(X_train_numeric_scaled):
        X_train_numeric_scaled = sp.csr_matrix(X_train_numeric_scaled)
    if not sp.issparse(X_val_numeric_scaled):
        X_val_numeric_scaled = sp.csr_matrix(X_val_numeric_scaled)

    # Combine preprocessed features
    X_train_processed = sp.hstack([X_train_numeric_scaled, X_train_categorical_encoded])
    X_val_processed = sp.hstack([X_val_numeric_scaled, X_val_categorical_encoded])

    return X_train_processed, X_val_processed

# Preprocess train and validation sets
X_train_processed, X_val_processed = preprocess_data(X_train, X_val)

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
class KNN:
    def __init__(self, k=5, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None

    def euclidean_distance(self, x1, x2):
        # Ensure inputs are 1D arrays
        x1 = x1.toarray().flatten() if sp.issparse(x1) else x1
        x2 = x2.toarray().flatten() if sp.issparse(x2) else x2
        return np.sqrt(np.sum((x1 - x2)**2))

    def manhattan_distance(self, x1, x2):
        x1 = x1.toarray().flatten() if sp.issparse(x1) else x1
        x2 = x2.toarray().flatten() if sp.issparse(x2) else x2
        return np.sum(np.abs(x1 - x2))

    def minkowski_distance(self, x1, x2, p=3):
        x1 = x1.toarray().flatten() if sp.issparse(x1) else x1
        x2 = x2.toarray().flatten() if sp.issparse(x2) else x2
        return np.power(np.sum(np.power(np.abs(x1 - x2), p)), 1/p)

    def fit(self, X, y):
        # Convert to dense if sparse
        self.X_train = X.toarray() if sp.issparse(X) else X
        self.y_train = y.reset_index(drop=True)

    def predict(self, X):
        # Convert to dense if sparse
        X_dense = X.toarray() if sp.issparse(X) else X
        predictions = [self._predict_single(x) for x in X_dense]
        return predictions

    def _predict_single(self, x):
        # Calculate distances
        if self.distance_metric == 'euclidean':
            distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]
        elif self.distance_metric == 'manhattan':
            distances = [self.manhattan_distance(x, x_train) for x_train in self.X_train]
        elif self.distance_metric == 'minkowski':
            distances = [self.minkowski_distance(x, x_train) for x_train in self.X_train]

        # Get k nearest neighbors
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Majority vote
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)

In [ ]:
knn = KNN(k=5, distance_metric='euclidean')
knn.fit(X_train_processed, y_train)

y_pred = knn.predict(X_val_processed)
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

With sklearn

In [ ]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    algorithm='auto',
    metric='euclidean'
)

knn.fit(X_train_processed, y_train)

y_pred = knn.predict(X_val_processed)

accuracy = accuracy_score(y_val, y_pred)
print(f"KNN Classifier Accuracy: {accuracy}")

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

KNN Classifier Accuracy: 0.9867526085253374

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.84      0.91      2152
           1       0.99      1.00      0.99     25929

    accuracy                           0.99     28081
   macro avg       0.99      0.92      0.95     28081
weighted avg       0.99      0.99      0.99     28081



## B. Naive Bayes

In [ ]:
class NaiveBayes:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]

        self.mean = np.zeros((n_classes, n_features))
        self.var = np.zeros((n_classes, n_features))
        self.priors = np.zeros(n_classes)

        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[i] = np.array(X_c.mean(axis=0)).ravel()
            X_c_squared = X_c.copy()
            X_c_squared.data **= 2
            self.var[i] = np.array(X_c_squared.mean(axis=0)).ravel() - self.mean[i] ** 2
            self.var[i] = np.maximum(self.var[i], self.var_smoothing)
            self.priors[i] = X_c.shape[0] / X.shape[0]

        return self

    def predict(self, X):
        X = X.toarray()
        y_pred = []
        for i in range(X.shape[0]):
            log_probs = []
            for j in range(len(self.classes)):
                log_prob = np.sum(-0.5 * np.log(2 * np.pi * self.var[j])
                                -0.5 * ((X[i] - self.mean[j]) ** 2) / self.var[j])
                log_prob += np.log(self.priors[j])
                log_probs.append(log_prob)
            y_pred.append(self.classes[np.argmax(log_probs)])

        return np.array(y_pred)

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

In [ ]:
nb = NaiveBayes()
nb.fit(X_train_processed, y_train)
y_pred = nb.predict(X_val_processed)
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

Accuracy: 0.11356433175456715

Classification Report:
              precision    recall  f1-score   support

           0       0.08      0.99      0.15      2152
           1       0.98      0.04      0.08     25929

    accuracy                           0.11     28081
   macro avg       0.53      0.52      0.11     28081
weighted avg       0.91      0.11      0.08     28081



with sk learn

In [ ]:
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train_processed.toarray(), y_train)
sk_predictions = gnb_classifier.predict(X_val_processed.toarray())

sk_accuracy = accuracy_score(y_val, sk_predictions)
print(f"Scikit-learn GaussianNB accuracy: {sk_accuracy}")
print("\nClassification Report (Scikit-learn GaussianNB):")
print(classification_report(y_val, sk_predictions))

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

## Does my model perform better in predicting one class than the other?
- Yes, some our model perform better in predicting one class than the other because the dataset has significant class imbalance.

## Which algorithm performs the best and what could be the reason?
- KNN Algorithm perform the best because KNN algorithm is able to recognize data patterns better than Naive Bayes. On the other hand, Naive Bayes algorithm is not suitable for big dimensional data modelling.

## Is it better for me to impute or drop the missing data?
- No, dropping data is even worse than imputing the data because it can reduce the accuracy. We prefer to take longer time to get more accurate results because each data can slightly give us important insight







